In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
# from registration_utilities import *

# import matplotlib.pyplot as plt
# %matplotlib inline

import time

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'

In [5]:
labels_twoSides = []
labels_twoSides_indices = {}
with open(volume_dir + 'annotationVolume_withOuterContour_labelIndices.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        name, index = line.split()
        labels_twoSides.append(name)
        labels_twoSides_indices[name] = int(index)
        
labelMap_sidedToUnsided = {name: name if '_' not in name else name[:-2] for name in labels_twoSides_indices.keys()}
labels_unsided = ['BackG'] + sorted(set(labelMap_sidedToUnsided.values()) - {'BackG', 'outerContour'}) + ['outerContour']
labels_unsided_indices = dict((j, i) for i, j in enumerate(labels_unsided))

In [8]:
# Compute Gradient

# for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
for stack in ['MD603']:
    
    print stack
    
    # numpy sparse does not support 3d
    section_bs_begin, section_bs_end = section_range_lookup[stack]

    (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax) = \
    np.loadtxt(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_limits.txt' % {'stack': stack}), dtype=np.int)
            
    volume2_allLabels = {}

    for name in labels_unsided:

        if name == 'BackG':
            continue

        volume2_roi = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_%(label)s.bp' % \
                                                          {'stack': stack, 'label': name})).astype(np.float16)
        volume2_allLabels[name] = volume2_roi
        del volume2_roi

    test_ydim, test_xdim, test_zdim = volume2_allLabels.values()[0].shape
    test_centroid = np.array([.5*test_xdim, .5*test_ydim, .5*test_zdim])

    t1 = time.time()

    for name in labels_unsided:

        if name == 'BackG':
            continue

        t = time.time()

        gy_gx_gz = np.gradient(volume2_allLabels[name].astype(np.float32), 3, 3, 3) 
        # 3.3 second, much faster than loading

        sys.stderr.write('gradient %s: %f seconds\n' % (name, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gy.bp' % {'stack':stack, 'label':name})
        bp.pack_ndarray_file(gy_gx_gz[1], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gx.bp' % {'stack':stack, 'label':name})
        bp.pack_ndarray_file(gy_gx_gz[2], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gz.bp' % {'stack':stack, 'label':name})

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

gradient 12N: 33.235280 seconds
save 12N: 1.053038 seconds
gradient 5N: 15.698635 seconds
save 5N: 2.585761 seconds
gradient 6N: 7.834003 seconds
save 6N: 1.227773 seconds
gradient 7N: 8.740844 seconds
save 7N: 2.180627 seconds
gradient 7n: 11.314858 seconds
save 7n: 0.864105 seconds
gradient AP: 7.511838 seconds
save AP: 1.076790 seconds
gradient Amb: 8.361379 seconds
save Amb: 1.061432 seconds
gradient LC: 7.836570 seconds
save LC: 1.783805 seconds
gradient LRt: 10.047722 seconds
save LRt: 2.485524 seconds
gradient Pn: 9.975672 seconds
save Pn: 3.642207 seconds
gradient R: 16.011557 seconds
save R: 1.906152 seconds
gradient RtTg: 8.573307 seconds
save RtTg: 3.963598 seconds
gradient Tz: 10.613999 seconds
save Tz: 1.347194 seconds
gradient VLL: 18.500608 seconds
save VLL: 3.243173 seconds
gradient sp5: 9.806077 seconds
save sp5: 3.180267 seconds
gradient outerContour: 11.688288 seconds
save outerContour: 2.745751 seconds


MD603


overall: 230.110624 seconds
